In [1]:
import torch
import sde_lib
import datasets
import sampling

from models import ddpm
from models import utils as mutils
import configs.default_cifar10_configs
import configs.vp.ddpm.cifar10_continuous

/storage/coda1/p-mtao8/0/krojas6/.conda/envs/fp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-05 14:22:42.670691: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/current/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-10.3.0/mvapich2-2.3.6-ouywalrqjnakjlhjxgunwqssb3iongrc/lib:/opt/slurm/current/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-4.8.5/libxml2-2.9.13-d4fgivwilgwkucth7kcsmpbcefmngewz/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-4.8.5/libiconv-1.16-pbdcxjpgrdv3k3eaisme7cw5fnl7zjmh/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-4.8.5/libpciacc

In [2]:

config = configs.vp.ddpm.cifar10_continuous.get_config()
# print(config)
N = 250
sde = sde_lib.VPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=N)
sampling_eps = 1e-3
score_model = mutils.create_model(config)
real_score_model = mutils.get_score_fn(sde,score_model,False,True)

In [3]:
train_ds, eval_ds, _ = datasets.get_dataset(config,
                                            uniform_dequantization=config.data.uniform_dequantization,
                                            evaluation=True)

2024-05-05 14:23:47.311770: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-05-05 14:23:47.374380: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/current/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-10.3.0/mvapich2-2.3.6-ouywalrqjnakjlhjxgunwqssb3iongrc/lib:/opt/slurm/current/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-4.8.5/libxml2-2.9.13-d4fgivwilgwkucth7kcsmpbcefmngewz/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-4.8.5/libiconv-1.16-pbdcxjpgrdv3k3eaisme7cw5fnl7zjmh/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-4.8.5/libpciaccess-0.16-wfowrnar2ry5ajqjs3pnjlhrlowivbxw/lib:/usr/local/pace-apps/spack/packages/linux-rhel7-x86_64/gcc-4.8.5/gcc-10.3.0-o57x6h2gubo7bzh7evmy4mvibdqrlghr/

In [4]:
sampling_shape = (1, config.data.num_channels,
                    config.data.image_size, config.data.image_size)
inverse_scaler = datasets.get_data_inverse_scaler(config)

sampling_fn = sampling.get_sampling_fn(config, sde, sampling_shape, inverse_scaler, sampling_eps)

samples, n = sampling_fn(score_model)
print(samples)

  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ts = torch.linspace(sampling_eps,1,device=device)

fp_loss = []
fp_loss_fn = sde.PDE(score_model).fp

for t in ts :
    cur_fp = 0
    for x,y in eval_ds:
        print(x,y)
        cur_fp += fp_loss_fn(x,t).detach().item()
    fp_loss.append(cur_fp)




In [ ]:
plt.plot(ts.detach().cpu().numpy(), fp_loss)
plt.show()